# DS-SF-30 | Codealong 03: Databases, Scrapping, and APIs; Part B - NoSQL Databases and `CouchDB` | Answer Key

## Administration Web Interface

> ## `CouchDB`'s Administration Web Interface, called `Fauxton` is at http://localhost:5984/_utils

## A brief overview of `CouchDB`'s bare-bone API using the command-line utility `curl`

> ### Hello World!

In [1]:
!curl -X GET http://localhost:5984

{"couchdb":"Welcome","version":"2.0.0","vendor":{"name":"The Apache Software Foundation"}}


We now know that `CouchDB` is alive.  Let's keep going.

> ### Get the list of databases

In [2]:
!curl -X GET http://localhost:5984/_all_dbs

[]


There are none so far.  So let's go and create one.

> ## Create a new database called `zillow`

In [3]:
!curl -X PUT http://localhost:5984/zillow

{"ok":true}


> ## Check out the new database

In [4]:
!curl -X GET http://localhost:5984/zillow

{"db_name":"zillow","update_seq":"0-g1AAAABXeJzLYWBgYMpgTmEQTM4vTc5ISXLIyU9OzMnILy7JAUklMiTV____PyuRAY-iPBYgydAApP5D1GYBAJmvHGw","sizes":{"file":8488,"external":0,"active":0},"purge_seq":0,"other":{"data_size":0},"doc_del_count":0,"doc_count":0,"disk_size":8488,"disk_format_version":6,"data_size":0,"compact_running":false,"instance_start_time":"0"}


It's a new database so it is empty (`"doc_count":0`).  Let's now populate it.

> ## Bulk import

In [5]:
!curl -d "@../datasets/dataset-03-zillow.json" -H "Content-Type: application/json" -X POST http://localhost:5984/zillow/_bulk_docs

[{"ok":true,"id":"15063471","rev":"1-7dce6f541ae7aa2a56ad7f413f2f2b74"},{"ok":true,"id":"15063505","rev":"1-591aba7a7dd407cfc71f1f4d6df0476c"},{"ok":true,"id":"15063609","rev":"1-ec0ae75cd2ca4208e98491a70aba1dd9"},{"ok":true,"id":"15064044","rev":"1-8770f8d482801645f7c9c60ea89244ec"},{"ok":true,"id":"15064257","rev":"1-c0f63878893eaccc8662d9b5c537350f"},{"ok":true,"id":"15064295","rev":"1-7f628d60ac75583f7118a0179a082ac9"},{"ok":true,"id":"15064391","rev":"1-574f556cb5df9ed7a8191331e97f29ee"},{"ok":true,"id":"15064536","rev":"1-115393bad4cbbd110ccb87b440dde7b2"},{"ok":true,"id":"15064640","rev":"1-bd23f2c92f0bbb3fea28389485a44130"},{"ok":true,"id":"15064669","rev":"1-99019cb2b30954fde35dfe8dc14769e2"},{"ok":true,"id":"15065032","rev":"1-202b5307a5c91bd8104662d1e6c40b5c"},{"ok":true,"id":"15065140","rev":"1-5e32c712477653a0ae2fdc88afb7acc8"},{"ok":true,"id":"15065727","rev":"1-3546034558c89806f0a1d232d7efc109"},{"ok":true,"id":"15065810","rev":"1-e3cef15689f998842cb5907d91939e21"},{"ok"

Let's check it again...

In [6]:
!curl -X GET http://localhost:5984/zillow

{"db_name":"zillow","update_seq":"1000-g1AAAABdeJzLYWBgYMpgTmEQTM4vTc5ISXLIyU9OzMnILy7JAUklMiTV____PyuJgYHxJx51eSxAkqEBSP2HKX-fBQBIsx5Y","sizes":{"file":450962,"external":359838,"active":437456},"purge_seq":0,"other":{"data_size":359838},"doc_del_count":0,"doc_count":1000,"disk_size":450962,"disk_format_version":6,"data_size":437456,"compact_running":false,"instance_start_time":"0"}


We can see that we now have 1,000 documents in it (`"doc_count":1000`).  Let's have a look at a couple of documents.

> ## `_id`

In [7]:
!curl -X GET http://localhost:5984/zillow/15065032

{"_id":"15065032","_rev":"1-202b5307a5c91bd8104662d1e6c40b5c","address":"101 Lombard St APT 116E, San Francisco, CA","map":{"latitude":37803470,"longitude":-122404090},"sale":{"date":"12/7/15","price":{"value":800000.0,"unit":"$"}},"is_a_studio":false,"beds":1,"baths":1.0,"size":{"value":1000.0,"unit":"sqft"},"built_in":{"year":1983}}


`15065032` refers to the Zillow identifier of our dataset but how did `CouchDB` know how to use it?  For the bulk import, we set the identifier of each document with `_id`.

## List of all documents within a database

In [8]:
!curl -X GET http://localhost:5984/zillow/_all_docs

> Now with the document contents

In [9]:
!curl -X GET http://localhost:5984/zillow/_all_docs?include_docs=true

{"total_rows":1000,"offset":0,"rows":[
{"id":"111710714","key":"111710714","value":{"rev":"1-71b3e5ffdc3e69eeb03e3e26a81d4bef"},"doc":{"_id":"111710714","_rev":"1-71b3e5ffdc3e69eeb03e3e26a81d4bef","address":"723 Taylor St # 501, San Francisco, CA","map":{"latitude":37789192,"longitude":-122412231},"sale":{"date":"12/2/15","price":{"value":798000.0,"unit":"$"}},"is_a_studio":false,"beds":1,"baths":1.0,"size":{"value":677.0,"unit":"sqft"},"built_in":{"year":2010}}},
{"id":"111711124","key":"111711124","value":{"rev":"1-9f3fe846c20063fd1995a1d84b28ad35"},"doc":{"_id":"111711124","_rev":"1-9f3fe846c20063fd1995a1d84b28ad35","address":"2125 Bryant St APT 5, San Francisco, CA","map":{"latitude":37760113,"longitude":-122409928},"sale":{"date":"11/17/15","price":{"value":1.38,"unit":"$M"}},"is_a_studio":false,"beds":2,"baths":2.0,"size":{"value":1240.0,"unit":"sqft"},"built_in":{"year":2009}}},
{"id":"111711143","key":"111711143","value":{"rev":"1-a78d553a42cd6641e5af78a09314aea9"},"doc":{"_id"

## Let's now using `CouchDB`'s `MapReduce` to compute the average sale price of these properties based on their number of bedrooms

> Using `Fauxton`, let's create a new view: http://localhost:5984/_utils/#/database/zillow/new_view

Copy/paste the following (JavaScript) code for the `Map function`.

```javascript
function(property) {
    if (property.beds) {
        beds = property.beds;
    } else if (property.is_a_studio == true) {
        beds = 0;
    } else {
        return;
    }
    if (property.sale.price.unit == '$') {
        emit(beds, [property.sale.price.value / Math.pow(10, 6), 1]);
        return;
    }
    if (property.sale.price.unit == '$M') {
        emit(beds, [property.sale.price.value, 1]);
        return;
    }
}
```

Copy/paste the following (JavaScript) code for the `Reduce (optional)` function (choose `CUSTOM`).

```javascript
function(keys, values) {
    weighted_price = 0
    weight = 0
        values.forEach(function (value) {
            weighted_price += value[0] * value[1];
            weight += value[1];
        });
     return [weighted_price / weight, weight];
}
```

Click on `Options` (upper right), check `Reduce` and `Run Query`.  Do these numbers match what we got from the `SQL` section?

## Delete a database

In [10]:
!curl -X DELETE http://localhost:5984/zillow

{"ok":true}
